In [1]:
import string

FILE = "american-english.txt"
ALLOWABLE_CHARACTERS = set(string.ascii_letters)
ALLOWED_ATTEMPTS = 6
WORD_LENGTH = 5

In [2]:
wordlist = [word.strip() for word in open(FILE, "r").readlines()]

#creating a set of valid words according to the rules of Wordle
WORDS = {
    word.lower()
    for word in wordlist    
    if len(word) == WORD_LENGTH and set(word) < ALLOWABLE_CHARACTERS
    }
WORDS

{'aeons',
 'piton',
 'elway',
 'issue',
 'zippy',
 'beebe',
 'erika',
 'elton',
 'condo',
 'intel',
 'viced',
 'luria',
 'niobe',
 'scold',
 'dunne',
 'susan',
 'bulks',
 'prove',
 'sloop',
 'gorky',
 'ocean',
 'keyed',
 'songs',
 'blank',
 'taper',
 'randi',
 'berms',
 'mitzi',
 'wrier',
 'abeam',
 'drips',
 'urals',
 'hubby',
 'keith',
 'sames',
 'hussy',
 'waive',
 'matte',
 'ranch',
 'serbs',
 'usage',
 'fitly',
 'borax',
 'furor',
 'ditch',
 'niter',
 'slosh',
 'amoco',
 'prone',
 'thieu',
 'heros',
 'awake',
 'dimly',
 'susie',
 'newer',
 'rolls',
 'silks',
 'clubs',
 'libya',
 'leave',
 'singh',
 'foxed',
 'tilde',
 'swill',
 'wagon',
 'ratio',
 'toffy',
 'damon',
 'chile',
 'gaunt',
 'guest',
 'watch',
 'cared',
 'nixes',
 'mousy',
 'viper',
 'width',
 'caped',
 'rises',
 'dobro',
 'stowe',
 'slabs',
 'piper',
 'lassa',
 'tyree',
 'jello',
 'fromm',
 'palsy',
 'verde',
 'orion',
 'ricks',
 'ibsen',
 'hurry',
 'overt',
 'pulpy',
 'hovel',
 'beard',
 'praia',
 'nazis',
 'belay',


In [3]:
from collections import Counter
from itertools import chain

#Creating a Counter Object that counts the number of occurences
#of each letter used across all valid Wordle words

LETTER_COUNTER = Counter(chain.from_iterable(WORDS))

LETTER_COUNTER


Counter({'a': 2691,
         'e': 3090,
         'o': 1877,
         'n': 1471,
         's': 2940,
         'p': 861,
         'i': 1728,
         't': 1517,
         'l': 1680,
         'w': 475,
         'y': 854,
         'u': 1040,
         'z': 158,
         'b': 733,
         'r': 1927,
         'k': 602,
         'c': 979,
         'd': 1203,
         'v': 336,
         'g': 682,
         'm': 861,
         'h': 828,
         'f': 502,
         'x': 117,
         'j': 130,
         'q': 48})

In [4]:
#finding the total occurances of all letters
total = len(WORDS)*WORD_LENGTH
LETTER_FREQUENCY = {character: value / total 
                    for character, value in LETTER_COUNTER.items()}

LETTER_FREQUENCY

{'a': 0.0917490623934538,
 'e': 0.10535288100920559,
 'o': 0.06399590862598023,
 'n': 0.05015342652574156,
 's': 0.10023866348448687,
 'p': 0.029355608591885442,
 'i': 0.05891578588475963,
 't': 0.051721786566655305,
 'l': 0.057279236276849645,
 'w': 0.016195022161609274,
 'y': 0.029116945107398567,
 'u': 0.035458574838049775,
 'z': 0.0053869757927037165,
 'b': 0.02499147630412547,
 'r': 0.06570064780088647,
 'k': 0.02052505966587112,
 'c': 0.033378793044664165,
 'd': 0.04101602454824412,
 'v': 0.011455847255369928,
 'g': 0.023252642345721105,
 'm': 0.029355608591885442,
 'h': 0.028230480736447322,
 'f': 0.017115581316058642,
 'x': 0.0039890896692806,
 'j': 0.004432321854756222,
 'q': 0.0016365496079099899}

In [5]:
def calculate_word_commonality(word):
    score = 0
    for char in word:
        score += LETTER_FREQUENCY[char]
    return score / (WORD_LENGTH - len(set(word)) + 1)

In [6]:
import operator

def sort_by_word_commonality(words):
    sort_by = operator.itemgetter(1)
    return sorted(
        [(word, calculate_word_commonality(word)) for word in words],
        key = sort_by,
        reverse=True,
    )

def display_word_table(word_commonalities):
    for (word,freq) in word_commonalities:
        print(f"{word:<10} | {freq:<5.2}")


In [7]:
def input_word():
    while True:
        word = input("Input the word you entered> ")
        if len(word) == WORD_LENGTH and word.lower() in WORDS:
            break
    return word.lower()

def input_response():
    print("Type the color-coded reply from Wordle:")
    print(" G for Green")
    print(" Y for Yellow")
    print(" ? for Gray")
    while True:
        response = input("Response from Wordle> ")
        if len(response) == WORD_LENGTH and set(response) <= {"G", "Y", "?"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

In [8]:
def match_word_vector(word, word_vector):
    assert len(word) == len(word_vector)
    for letter, v_letter in zip(word, word_vector):
        if letter not in v_letter:
            return False
    return True

def match(word_vector, possible_words):
    return [word for word in possible_words if match_word_vector(word, word_vector)]

In [9]:
def solve():
    possible_words = WORDS.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(WORD_LENGTH)]
    for attempt in range(1, ALLOWED_ATTEMPTS + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        display_word_table(sort_by_word_commonality(possible_words)[:15])
        word = input_word()
        response = input_response()
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [ ]:
solve()

Attempt 1 with 5866 possible words
arose      | 0.43 
arise      | 0.42 
raise      | 0.42 
aries      | 0.42 
reals      | 0.42 
earls      | 0.42 
laser      | 0.42 
aloes      | 0.42 
stare      | 0.41 
tears      | 0.41 
aster      | 0.41 
rates      | 0.41 
tares      | 0.41 
aisle      | 0.41 
elisa      | 0.41 
Input the word you entered> AROSE
Type the color-coded reply from Wordle:
 G for Green
 Y for Yellow
 ? for Gray
Response from Wordle> YYYY?
Attempt 2 with 2348 possible words
solar      | 0.38 
liars      | 0.37 
lairs      | 0.37 
liras      | 0.37 
rails      | 0.37 
sorta      | 0.37 
taros      | 0.37 
sarto      | 0.37 
sonar      | 0.37 
roans      | 0.37 
stair      | 0.37 
sitar      | 0.37 
rains      | 0.37 
iotas      | 0.37 
snarl      | 0.37 
Input the word you entered> SOLAR
Type the color-coded reply from Wordle:
 G for Green
 Y for Yellow
 ? for Gray
